In [1]:
train_path = '../../BristolGorillas2020/images/train'
test_path = '../../BristolGorillas2020/images/test'

In [2]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [10]:
image_paths = []
image_files = []
image_annotations = []

for path, subdirs, files in os.walk(train_path):
    for name in files:
        if(name.endswith('.jpg')):
            data_point = {}
            data_point['file_name'] = name
            data_point['file_path'] = (os.path.join(path, name))
            data_point['class'] = name.split('-')[0]
            image_paths.append(data_point)
            
        elif(name.endswith('.png')):
            data_point = {}
            data_point['file_name'] = name
            data_point['file_path'] = (os.path.join(path, name))
            data_point['class'] = name.split('-')[0]
            image_paths.append(data_point)
            
        elif(name.endswith('.txt')):
            data_point = {}
            with open(os.path.join(path, name),"r") as label:
                data_point['annotation_file'] = name
                data_point['annotation_path'] = (os.path.join(path, name))
                annotation = label.readlines()
                annotation = [x.rstrip('\n') for x in annotation]
                for i in range(len(annotation)):
                    data_point['annotation_' + str(i)] = annotation[i]
            image_annotations.append(data_point)
        else:
            pass

In [11]:
x = pd.DataFrame(image_paths)

In [12]:
train_data = pd.DataFrame(image_annotations)

In [13]:
train_data.insert(0,'file_name',pd.Series(x['file_name']))
train_data.insert(2,'image_path',pd.Series(x['file_path']))

In [16]:
train_data.head()

,file_name,annotation_file,image_path,annotation_path,annotation_0,annotation_1,annotation_2
0,ayana-1-img-1.jpg,ayana-1-img-1.txt,../../BristolGorillas2020/images/train/ayana/a...,../../BristolGorillas2020/images/train/ayana/a...,1 0.557292 0.212500 0.026042 0.036111,NaN,NaN
1,ayana-1-img-2.jpg,ayana-1-img-2.txt,../../BristolGorillas2020/images/train/ayana/a...,../../BristolGorillas2020/images/train/ayana/a...,1 0.914844 0.618981 0.167187 0.308333,NaN,NaN
2,ayana-1-img-3.jpg,ayana-1-img-3.txt,../../BristolGorillas2020/images/train/ayana/a...,../../BristolGorillas2020/images/train/ayana/a...,1 0.657031 0.431481 0.030729 0.079630,NaN,NaN
3,ayana-1-img-4.jpg,ayana-1-img-4.txt,../../BristolGorillas2020/images/train/ayana/a...,../../BristolGorillas2020/images/train/ayana/a...,1 0.473438 0.503241 0.023958 0.058333,NaN,NaN
4,ayana-1-img-5.jpg,ayana-1-img-5.txt,../../BristolGorillas2020/images/train/ayana/a...,../../BristolGorillas2020/images/train/ayana/a...,1 0.575521 0.295370 0.044792 0.083333,NaN,NaN


In [13]:
train_data.to_csv('bg2020_train.csv')

In [17]:
train_data['annotation_0'].isnull().values.any()

True

In [18]:
train_data['annotation_0'].isnull().sum()

6